In [1]:
import sys
import os

%load_ext autoreload
%autoreload 2

sys.path.append('../')
sys.path.insert(0, os.path.dirname(os.getcwd()) + "/../")

print(sys.path)

['/home/hieutt/CAN-SupCon-IDS/../', '/home/hieutt/CAN-SupCon-IDS/notebooks', '/home/hieutt/miniconda3/envs/torchtf/lib/python39.zip', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [3]:
import pandas as pd
# import vaex
import numpy as np
import glob
import dask.dataframe as dd
import json
from sklearn.model_selection import train_test_split
import math
import csv
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import time
import _warnings
import tensorflow as tf
from tqdm import tqdm
import swifter
import argparse
import util
from importlib import reload

In [8]:
os.chdir(os.path.dirname(os.getcwd()) + "/data/road/attacks")
# os.chdir(os.path.dirname(os.getcwd()) + "/road/ambient")
print(os.getcwd())

/home/hieutt/CAN-SupCon-IDS/Data/road/attacks


# Process ROAD log datasets into CSV

In [10]:
def get_all_data(attack_dict):
    df_aggregation = []
    
    for attack_name, metadata in attack_dict.items():    
        if "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"{attack_name}")
            file_name = '/home/hieutt/CAN-SupCon-IDS/Data/road/attacks/{}.log'.format(attack_name)
            df_attack = util.make_can_df(file_name)
            df_attack = util.add_time_diff_per_aid_col(df_attack)
            # print(df_attack.shape)
            # print(df.dtypes)
            df_aggregation.append(df_attack)
            print(f"Finish preprocess {file_name}")
    return df_aggregation

def get_time_interval(attack_dict):
    attack_metadata = []
    
    for attack_name, metadata in attack_dict.items():    
        if "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"Finish get time interval of {attack_name}")
            
            # From metadata file
            attack_metadata.append([tuple(attack_dict[attack_name]["injection_interval"])])
    return attack_metadata

def mark_label(df_aggregation, attack_metadata, attack_dict):
    count = 0
    for attack_name, metadata in attack_dict.items():    
        if "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"Index {count}: {attack_name} --- {attack_dict[attack_name]['injection_id']}")
            
            if attack_dict[attack_name]["injection_id"] != "XXX":
                df_aggregation[count] = util.add_actual_attack_col(df_aggregation[count], attack_metadata[count], int(attack_dict[attack_name]["injection_id"], 16), attack_dict[attack_name]["injection_data_str"], attack_name)
                print(len(df_aggregation[count][df_aggregation[count]['label'] == True]['label']))
                print(len(df_aggregation[count][df_aggregation[count]['label'] == False]['label']))
            else:
                df_aggregation[count] = util.add_actual_attack_col(df_aggregation[count], attack_metadata[count], "XXX", attack_dict[attack_name]["injection_data_str"], attack_name)
                print(len(df_aggregation[count][df_aggregation[count]['label'] == True]['label']))
                print(len(df_aggregation[count][df_aggregation[count]['label'] == False]['label']))
            count += 1
    return df_aggregation

def filter_attack(arr, keyword):
    sub_array = []
    for item in arr:
        if keyword in item:
            sub_array.append(item)
    return sub_array

In [11]:
with open("/home/hieutt/Unicon/data/road/attacks/capture_metadata.json", "r") as read_file:
    attack_dict = json.load(read_file)
# display(attack_dict)

df_aggregation = get_all_data(attack_dict)
attack_metadata = get_time_interval(attack_dict)
df_aggregation = mark_label(df_aggregation, attack_metadata, attack_dict)

display(df_aggregation[0])

correlated_signal_attack_1
Finish preprocess /home/hieutt/CAN-SupCon-IDS/Data/road/attacks/correlated_signal_attack_1.log
correlated_signal_attack_1_masquerade
Finish preprocess /home/hieutt/CAN-SupCon-IDS/Data/road/attacks/correlated_signal_attack_1_masquerade.log
correlated_signal_attack_2
Finish preprocess /home/hieutt/CAN-SupCon-IDS/Data/road/attacks/correlated_signal_attack_2.log
correlated_signal_attack_2_masquerade
Finish preprocess /home/hieutt/CAN-SupCon-IDS/Data/road/attacks/correlated_signal_attack_2_masquerade.log
correlated_signal_attack_3
Finish preprocess /home/hieutt/CAN-SupCon-IDS/Data/road/attacks/correlated_signal_attack_3.log
correlated_signal_attack_3_masquerade
Finish preprocess /home/hieutt/CAN-SupCon-IDS/Data/road/attacks/correlated_signal_attack_3_masquerade.log
fuzzing_attack_1
Finish preprocess /home/hieutt/CAN-SupCon-IDS/Data/road/attacks/fuzzing_attack_1.log
fuzzing_attack_2
Finish preprocess /home/hieutt/CAN-SupCon-IDS/Data/road/attacks/fuzzing_attack_2.lo

,time,aid,data,time_diffs,label
3873,1.618163,6,0800006400000000,0.999845,False
6266,2.618064,6,0800006400000000,0.999901,False
8655,3.617806,6,0800006400000000,0.999742,False
11049,4.617810,6,0800006400000000,1.000004,False
13441,5.618164,6,0800006400000000,1.000354,False
...,...,...,...,...,...
80160,32.636582,1788,00000738D2B85800,0.099969,False
80402,32.736450,1788,0000073732B86000,0.099868,False
80641,32.836612,1788,0000073892B86800,0.100162,False
80874,32.936532,1788,00000737B2B87000,0.099920,False


In [13]:
count = 0
out_mas = '/home/hieutt/Unicon/data/road/mas_dataset/'
out_fab = '/home/hieutt/Unicon/data/road/fab_dataset/'
for attack_name, metadata in attack_dict.items():
    if "accelerator" not in attack_name and "metadata" not in attack_name:
        print(f"Saving {attack_name}_dataset.csv")
        if "masquerade" not in attack_name:
            foutput = '{}/{}_dataset.csv'.format(out_fab, attack_name)
            df_aggregation[count].to_csv(foutput, index=False)
        else:
            foutput = '{}/{}_dataset.csv'.format(out_mas, attack_name)
            df_aggregation[count].to_csv(foutput, index=False)
        count += 1
        

Saving correlated_signal_attack_1_dataset.csv
Saving correlated_signal_attack_1_masquerade_dataset.csv
Saving correlated_signal_attack_2_dataset.csv
Saving correlated_signal_attack_2_masquerade_dataset.csv
Saving correlated_signal_attack_3_dataset.csv
Saving correlated_signal_attack_3_masquerade_dataset.csv
Saving fuzzing_attack_1_dataset.csv
Saving fuzzing_attack_2_dataset.csv
Saving fuzzing_attack_3_dataset.csv
Saving max_engine_coolant_temp_attack_dataset.csv
Saving max_engine_coolant_temp_attack_masquerade_dataset.csv
Saving max_speedometer_attack_1_dataset.csv
Saving max_speedometer_attack_1_masquerade_dataset.csv
Saving max_speedometer_attack_2_dataset.csv
Saving max_speedometer_attack_2_masquerade_dataset.csv
Saving max_speedometer_attack_3_dataset.csv
Saving max_speedometer_attack_3_masquerade_dataset.csv
Saving reverse_light_off_attack_1_dataset.csv
Saving reverse_light_off_attack_1_masquerade_dataset.csv
Saving reverse_light_off_attack_2_dataset.csv
Saving reverse_light_off_a